In [1]:
pip install PyMuPDF docx2txt

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 13.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 50.3 MB/s eta 0:00:0000:0100:01
  Created wheel for docx2txt: filename=docx2txt-0.8-py3-none-any.whl size=3960 sha256=ff48ffabb48b5c33a59cf83c2f1c78e6e7a8de5d3dc14632351e3bb7e9f32c40
  Stored in directory: /root/.cache/pip/wheels/22/58/cf/093d0a6c3ecfdfc5f6ddd5524043b88e59a9a199cb02352966
Successfully built docx2txt
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import re
import fitz  
import docx2txt
import pandas as pd
from openpyxl.utils.exceptions import IllegalCharacterError

In [3]:
RESUME_DIR = '/kaggle/input/resumes'

In [4]:
def load_pdf_text(file_path):
    doc = fitz.open(file_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text.splitlines()

In [5]:
def load_docx_text(file_path):
    text = docx2txt.process(file_path)
    return text.splitlines()

In [6]:
def preprocess_text(lines):
    def remove_punctuation(line):
        return re.sub(r'[.,]', '', line)
    processed_lines = []
    for line in lines:
        line = line.lower().strip()
        line = remove_punctuation(line)
        processed_lines.append(' '.join(line.split()))
    return processed_lines

In [7]:
SKILL_TERMS = [
    "python", "Python", "PYTHON",
    "java", "Java", "JAVA",
    "c++", "C++", "cpp", "CPP", 
    "c#", "C#", "csharp", "Csharp", "CSHARP",
    "javascript", "JavaScript", "JAVASCRIPT", "js", "JS",
    "html", "HTML", "html5", "HTML5", 
    "css", "CSS", "css3", "CSS3",
    "sql", "SQL", 
    "r", "R", "r programming", "R programming", "R Programming",
    "ruby", "Ruby", "RUBY", 
    "php", "PHP", "Php",
    "swift", "Swift", "SWIFT", 
    "go", "Go", "GO", "golang", "Golang", "GOLANG",
    "kotlin", "Kotlin", "KOTLIN",
    "bash", "Bash", "BASH", 
    "perl", "Perl", "PERL", 
    "matlab", "MATLAB", "MatLab",
    "scala", "Scala", "SCALA",
    "html5", "HTML5",
    "css3", "CSS3",
    "typescript", "TypeScript", "TYPESCRIPT", "ts", "TS",
    "react", "React", "REACT", "react.js", "React.js", "REACT.JS",
    "angular", "Angular", "ANGULAR", "angular.js", "Angular.js", "ANGULAR.JS",
    "vue.js", "Vue.js", "VUE.JS", "vuejs", "Vuejs", "VUEJS",
    "node.js", "Node.js", "NODE.JS", "nodejs", "Nodejs", "NODEJS",
    "django", "Django", "DJANGO", 
    "flask", "Flask", "FLASK", 
    "bootstrap", "Bootstrap", "BOOTSTRAP",
    "jquery", "jQuery", "JQUERY",
    "asp.net", "ASP.NET", "aspnet", "AspNet", "ASPNet",
    "pandas", "Pandas", "PANDAS",
    "numpy", "NumPy", "NUMPY",
    "scipy", "SciPy", "SCIPY",
    "scikit-learn", "Scikit-learn", "SCIKIT-LEARN", "sklearn", "Sklearn", "SKLEARN",
    "tensorflow", "TensorFlow", "TENSORFLOW", 
    "keras", "Keras", "KERAS", 
    "pytorch", "PyTorch", "PYTORCH", 
    "matplotlib", "Matplotlib", "MATPLOTLIB", 
    "seaborn", "Seaborn", "SEABORN",
    "jupyter", "Jupyter", "JUPYTER", "jupyter notebook", "Jupyter Notebook", "JUPYTER NOTEBOOK",
    "rstudio", "RStudio", "RSTUDIO",
    "tableau", "Tableau", "TABLEAU",
    "power bi", "Power BI", "POWER BI", "PowerBI", "POWERBI",
    "sas", "SAS", "Sas",
    "spss", "SPSS", "Spss",
    "hadoop", "Hadoop", "HADOOP",
    "spark", "Spark", "SPARK",
    "hive", "Hive", "HIVE", 
    "etl", "ETL", "Etl",
    "data warehousing", "Data Warehousing", "DATA WAREHOUSING",
    "data mining", "Data Mining", "DATA MINING",
    "nlp", "NLP", "Nlp", "natural language processing", "Natural Language Processing", "NATURAL LANGUAGE PROCESSING",
    "deep learning", "Deep Learning", "DEEP LEARNING", 
    "aws", "AWS", "Aws", "amazon web services", "Amazon Web Services", "AMAZON WEB SERVICES",
    "azure", "Azure", "AZURE", 
    "google cloud platform", "Google Cloud Platform", "GOOGLE CLOUD PLATFORM", "gcp", "GCP", "Gcp",
    "docker", "Docker", "DOCKER", 
    "kubernetes", "Kubernetes", "KUBERNETES", 
    "openshift", "OpenShift", "OPENSHIFT", 
    "terraform", "Terraform", "TERRAFORM", 
    "ansible", "Ansible", "ANSIBLE", 
    "jenkins", "Jenkins", "JENKINS", 
    "ci/cd", "CI/CD", "Ci/Cd", "CICD", 
    "devops", "DevOps", "DEVOPS", "dev ops", "Dev Ops", "DEV OPS", 
    "cloudformation", "CloudFormation", "CLOUDFORMATION", 
    "mysql", "MySQL", "MYSQL", 
    "postgresql", "PostgreSQL", "POSTGRESQL", 
    "mongodb", "MongoDB", "MONGODB", 
    "oracle", "Oracle", "ORACLE", 
    "sqlite", "SQLite", "SQLITE", 
    "redis", "Redis", "REDIS", 
    "cassandra", "Cassandra", "CASSANDRA", 
    "elasticsearch", "Elasticsearch", "ELASTICSEARCH", 
    "nosql", "NoSQL", "NOSQL", "no sql", "No SQL", "NO SQL",
    "sql server", "SQL Server", "SQL SERVER", 
    "mariadb", "MariaDB", "MARIADB", 
    "data modeling", "Data Modeling", "DATA MODELING", 
    "database design", "Database Design", "DATABASE DESIGN", 
    "git", "Git", "GIT", 
    "github", "GitHub", "GITHUB", 
    "gitlab", "GitLab", "GITLAB", 
    "bitbucket", "BitBucket", "BITBUCKET", 
    "svn", "SVN", "Svn",
    "mercurial", "Mercurial", "MERCURIAL", 
    "android", "Android", "ANDROID", 
    "ios", "iOS", "IOS", 
    "react native", "React Native", "REACT NATIVE", 
    "flutter", "Flutter", "FLUTTER", 
    "swift", "Swift", "SWIFT", 
    "objective-c", "Objective-C", "OBJECTIVE-C", "objective c", "Objective C", "OBJECTIVE C", 
    "kotlin", "Kotlin", "KOTLIN", 
    "xcode", "XCode", "XCODE", 
    "android studio", "Android Studio", "ANDROID STUDIO", 
    "tcp/ip", "TCP/IP", "Tcp/Ip", 
    "dns", "DNS", "Dns", 
    "vpn", "VPN", "Vpn", 
    "firewall", "Firewall", "FIREWALL", 
    "cybersecurity", "Cybersecurity", "CYBERSECURITY", "cyber security", "Cyber Security", "CYBER SECURITY",
    "network security", "Network Security", "NETWORK SECURITY", 
    "ssl/tls", "SSL/TLS", "Ssl/Tls", 
    "encryption", "Encryption", "ENCRYPTION", 
    "penetration testing", "Penetration Testing", "PENETRATION TESTING", 
    "ethical hacking", "Ethical Hacking", "ETHICAL HACKING", 
    "owasp", "OWASP", "Owasp", 
    "iso 27001", "ISO 27001", "Iso 27001", 
    "soc 2", "SOC 2", "Soc 2", 
    "network monitoring", "Network Monitoring", "NETWORK MONITORING", 
    "performance optimization", "Performance Optimization", "PERFORMANCE OPTIMIZATION", 
    "configuration and management of routers", "Configuration And Management Of Routers", "CONFIGURATION AND MANAGEMENT OF ROUTERS", 
    "switches", "Switches", "SWITCHES", 
    "firewalls", "Firewalls", "FIREWALLS", 
    "network infrastructure", "Network Infrastructure", "NETWORK INFRASTRUCTURE", 
    "remote technical support", "Remote Technical Support", "REMOTE TECHNICAL SUPPORT", 
    "windows", "Windows", "WINDOWS", 
    "linux", "Linux", "LINUX", 
    "macos", "macOS", "MACOS", 
    "unix", "Unix", "UNIX", 
    "virtualization", "Virtualization", "VIRTUALIZATION", 
    "vmware", "VMware", "VMWARE", 
    "hyper-v", "Hyper-V", "HYPER-V", 
    "esxi", "ESXi", "ESXI", 
    "server virtualization", "Server Virtualization", "SERVER VIRTUALIZATION", 
    "vcp-dcv", "VCP-DCV", "Vcp-Dcv", 
    "vmware esxi", "VMware ESXi", "VMWARE ESXI", 
    "vmware vcenter", "VMware vCenter", "VMWARE VCENTER", 
    "server management", "Server Management", "SERVER MANAGEMENT", 
    "hpe server administration", "HPE Server Administration", "HPE SERVER ADMINISTRATION", 
    "microsoft windows server", "Microsoft Windows Server", "MICROSOFT WINDOWS SERVER", 
    "microsoft windows server installation", "Microsoft Windows Server Installation", "MICROSOFT WINDOWS SERVER INSTALLATION", 
    "configuration", "Configuration", "CONFIGURATION", 
    "administration", "Administration", "ADMINISTRATION", 
    "security hardening", "Security Hardening", "SECURITY HARDENING", 
    "backup and restore", "Backup and Restore", "BACKUP AND RESTORE", 
    "implementation and management of backup and recovery solutions", "Implementation and Management of Backup and Recovery Solutions", "IMPLEMENTATION AND MANAGEMENT OF BACKUP AND RECOVERY SOLUTIONS", 
    "os image creation", "OS Image Creation", "OS IMAGE CREATION", 
    "hardware troubleshooting", "Hardware Troubleshooting", "HARDWARE TROUBLESHOOTING", 
    "backup tools", "Backup Tools", "BACKUP TOOLS", 
    "veeam backup tool", "Veeam Backup Tool", "VEEAM BACKUP TOOL", 
    "cloud services", "Cloud Services", "CLOUD SERVICES", 
    "microsoft 365", "Microsoft 365", "MICROSOFT 365", 
    "macos troubleshooting", "macOS Troubleshooting", "MACOS TROUBLESHOOTING", 
    "apple devices support", "Apple Devices Support", "APPLE DEVICES SUPPORT", 
    "hardware troubleshooting", "Hardware Troubleshooting", "HARDWARE TROUBLESHOOTING", 
    "software troubleshooting", "Software Troubleshooting", "SOFTWARE TROUBLESHOOTING", 
    "hardware & software troubleshooting", "Hardware & Software Troubleshooting", "HARDWARE & SOFTWARE TROUBLESHOOTING", 
    "diagnosing", "Diagnosing", "DIAGNOSING", 
    "repairing issues", "Repairing Issues", "REPAIRING ISSUES", 
    "servers", "Servers", "SERVERS", 
    "desktops", "Desktops", "DESKTOPS", 
    "laptops", "Laptops", "LAPTOPS", 
    "printers", "Printers", "PRINTERS", 
    "winbox", "Winbox", "WINBOX", 
    "kaspersky", "Kaspersky", "KASPERSKY", 
    "ticket system", "Ticket System", "TICKET SYSTEM", 
    "fluke tests", "Fluke Tests", "FLUKE TESTS", 
    "surveillance systems", "Surveillance Systems", "SURVEILLANCE SYSTEMS", 
    "voip", "VoIP", "VOIP", 
    "communication", "Communication", "COMMUNICATION", 
    "leadership", "Leadership", "LEADERSHIP", 
    "teamwork", "Teamwork", "TEAMWORK", 
    "problem-solving", "Problem-Solving", "PROBLEM-SOLVING", "problem solving", "Problem Solving", "PROBLEM SOLVING", 
    "creativity", "Creativity", "CREATIVITY", 
    "time management", "Time Management", "TIME MANAGEMENT", 
    "adaptability", "Adaptability", "ADAPTABILITY", 
    "customer service", "Customer Service", "CUSTOMER SERVICE", 
    "remote technical support", "Remote Technical Support", "REMOTE TECHNICAL SUPPORT", 
    "financial analysis", "Financial Analysis", "FINANCIAL ANALYSIS", 
    "accounting", "Accounting", "ACCOUNTING", 
    "quickbooks", "QuickBooks", "QUICKBOOKS", 
    "sap", "SAP", "Sap", 
    "financial modeling", "Financial Modeling", "FINANCIAL MODELING", 
    "budgeting", "Budgeting", "BUDGETING", 
    "auditing", "Auditing", "AUDITING", 
    "taxation", "Taxation", "TAXATION", 
    "investment banking", "Investment Banking", "INVESTMENT BANKING", 
    "portfolio management", "Portfolio Management", "PORTFOLIO MANAGEMENT", 
    "patient care", "Patient Care", "PATIENT CARE", 
    "emr", "EMR", "Emr", 
    "ehr", "EHR", "Ehr", 
    "medical coding", "Medical Coding", "MEDICAL CODING", 
    "hipaa", "HIPAA", "Hipaa", 
    "clinical research", "Clinical Research", "CLINICAL RESEARCH", 
    "pharmacy", "Pharmacy", "PHARMACY", 
    "nursing", "Nursing", "NURSING", 
    "medical terminology", "Medical Terminology", "MEDICAL TERMINOLOGY", 
    "seo", "SEO", "Seo", 
    "sem", "SEM", "Sem", 
    "social media marketing", "Social Media Marketing", "SOCIAL MEDIA MARKETING", 
    "content marketing", "Content Marketing", "CONTENT MARKETING", 
    "google analytics", "Google Analytics", "GOOGLE ANALYTICS", 
    "google ads", "Google Ads", "GOOGLE ADS", 
    "email marketing", "Email Marketing", "EMAIL MARKETING", 
    "copywriting", "Copywriting", "COPYWRITING", 
    "branding", "Branding", "BRANDING", 
    "market research", "Market Research", "MARKET RESEARCH", 
    "crm", "CRM", "Crm", 
    "sales strategy", "Sales Strategy", "SALES STRATEGY", 
    "lead generation", "Lead Generation", "LEAD GENERATION", 
    "negotiation", "Negotiation", "NEGOTIATION", 
    "b2b sales", "B2B Sales", "B2B SALES", 
    "b2c sales", "B2C Sales", "B2C SALES", 
    "sales forecasting", "Sales Forecasting", "SALES FORECASTING", 
    "pipeline management", "Pipeline Management", "PIPELINE MANAGEMENT", 
    "curriculum development", "Curriculum Development", "CURRICULUM DEVELOPMENT", 
    "instructional design", "Instructional Design", "INSTRUCTIONAL DESIGN", 
    "classroom management", "Classroom Management", "CLASSROOM MANAGEMENT", 
    "e-learning", "E-Learning", "E-LEARNING", 
    "educational technology", "Educational Technology", "EDUCATIONAL TECHNOLOGY", 
    "lesson planning", "Lesson Planning", "LESSON PLANNING", 
    "graphic design", "Graphic Design", "GRAPHIC DESIGN", 
    "adobe photoshop", "Adobe Photoshop", "ADOBE PHOTOSHOP", 
    "illustrator", "Illustrator", "ILLUSTRATOR", 
    "indesign", "InDesign", "INDESIGN", 
    "figma", "Figma", "FIGMA", 
    "sketch", "Sketch", "SKETCH", 
    "coreldraw", "CorelDraw", "CORELDRAW", 
    "canva", "Canva", "CANVA", 
    "video editing", "Video Editing", "VIDEO EDITING", 
    "audio editing", "Audio Editing", "AUDIO EDITING", 
    "adobe premiere", "Adobe Premiere", "ADOBE PREMIERE", 
    "final cut pro", "Final Cut Pro", "FINAL CUT PRO", 
    "after effects", "After Effects", "AFTER EFFECTS", 
    "audacity", "Audacity", "AUDACITY", 
    "logic pro", "Logic Pro", "LOGIC PRO", 
    "ux/ui design", "UX/UI Design", "UX/UI DESIGN", "UX UI Design", "UXUI Design", "UX-UI Design",
    "wireframing", "Wireframing", "WIREFRAMING", 
    "prototyping", "Prototyping", "PROTOTYPING", 
    "user research", "User Research", "USER RESEARCH", 
    "interaction design", "Interaction Design", "INTERACTION DESIGN", 
    "information architecture", "Information Architecture", "INFORMATION ARCHITECTURE", 
    "3d modeling", "3D Modeling", "3D MODELING", 
    "autocad", "AutoCAD", "AUTOCAD", 
    "solidworks", "SolidWorks", "SOLIDWORKS", 
    "blender", "Blender", "BLENDER", 
    "3ds max", "3ds Max", "3DS MAX", 
    "rhino", "Rhino", "RHINO", 
    "maya", "Maya", "MAYA", 
    "agile", "Agile", "AGILE", 
    "scrum", "Scrum", "SCRUM", 
    "kanban", "Kanban", "KANBAN", 
    "waterfall", "Waterfall", "WATERFALL", 
    "pmp", "PMP", "Pmp", 
    "jira", "JIRA", "Jira", 
    "trello", "Trello", "TRELLO", 
    "ms project", "MS Project", "MS PROJECT", 
    "asana", "Asana", "ASANA", 
    "risk management", "Risk Management", "RISK MANAGEMENT", 
    "stakeholder management", "Stakeholder Management", "STAKEHOLDER MANAGEMENT", 
    "resource allocation", "Resource Allocation", "RESOURCE ALLOCATION", 
    "gantt charts", "Gantt Charts", "GANTT CHARTS", 
    "qa", "QA", "Qa", 
    "quality assurance", "Quality Assurance", "QUALITY ASSURANCE", 
    "test automation", "Test Automation", "TEST AUTOMATION", 
    "selenium", "Selenium", "SELENIUM", 
    "manual testing", "Manual Testing", "MANUAL TESTING", 
    "automated testing", "Automated Testing", "AUTOMATED TESTING", 
    "regression testing", "Regression Testing", "REGRESSION TESTING", 
    "unit testing", "Unit Testing", "UNIT TESTING", 
    "integration testing", "Integration Testing", "INTEGRATION TESTING", 
    "continuous integration", "Continuous Integration", "CONTINUOUS INTEGRATION", 
    "continuous deployment", "Continuous Deployment", "CONTINUOUS DEPLOYMENT", 
    "tdd", "TDD", "Tdd", "test-driven development", "Test-Driven Development", "TEST-DRIVEN DEVELOPMENT", 
    "bdd", "BDD", "Bdd", "behavior-driven development", "Behavior-Driven Development", "BEHAVIOR-DRIVEN DEVELOPMENT"
]


In [8]:
EDUCATION_TERMS = [
    "bachelor", "Bachelor", "BACHELOR", "bachelor's", "Bachelor's", "BACHELOR'S", 
    "master", "Master", "MASTER", "master's", "Master's", "MASTER'S",
    "phd", "PhD", "PHD", "Ph.D.", "PH.D.", "ph.d.", 
    "associate", "Associate", "ASSOCIATE", 
    "diploma", "Diploma", "DIPLOMA", 
    "certificate", "Certificate", "CERTIFICATE", 
    "degree", "Degree", "DEGREE", 
    "graduate", "Graduate", "GRADUATE", 
    "postgraduate", "Postgraduate", "POSTGRADUATE", "post-graduate", "Post-graduate", "POST-GRADUATE",
    "undergraduate", "Undergraduate", "UNDERGRADUATE", "under-graduate", "Under-graduate", "UNDER-GRADUATE", 
    "bsc", "BSc", "BSC", "B.Sc.", "B.SC.", 
    "msc", "MSc", "MSC", "M.Sc.", "M.SC.",
    "ba", "BA", "B.A.", "B.A", "BA.", 
    "ma", "MA", "M.A.", "M.A", "MA.",
    "bs", "BS", "B.S.", "B.S", "BS.",
    "ms", "MS", "M.S.", "M.S", "MS.",
    "mba", "MBA", "M.B.A.", "M.B.A", "MBA.",
    "m.tech", "M.Tech", "M.TECH", "Master of Technology", "MASTER OF TECHNOLOGY", "MTech", "MTECH", 
    "b.tech", "B.Tech", "B.TECH", "Bachelor of Technology", "BACHELOR OF TECHNOLOGY", "BTech", "BTECH",
    "doctorate", "Doctorate", "DOCTORATE", "doctoral degree", "Doctoral Degree", "DOCTORAL DEGREE", 
    "doctor of philosophy", "Doctor of Philosophy", "DOCTOR OF PHILOSOPHY", 
    "jd", "JD", "J.D.", "J.D", "Juris Doctor", "JURIS DOCTOR", 
    "md", "MD", "M.D.", "M.D", "Doctor of Medicine", "DOCTOR OF MEDICINE", 
    "pharmd", "PharmD", "PHARMD", "Doctor of Pharmacy", "DOCTOR OF PHARMACY", 
    "engineering", "Engineering", "ENGINEERING",
    "law", "Law", "LAW", 
    "medicine", "Medicine", "MEDICINE", 
    "science", "Science", "SCIENCE", 
    "arts", "Arts", "ARTS", 
    "business", "Business", "BUSINESS", 
    "technology", "Technology", "TECHNOLOGY", 
    "information technology", "Information Technology", "INFORMATION TECHNOLOGY", "IT", "It", "it", 
    "computer science", "Computer Science", "COMPUTER SCIENCE", 
    "electrical engineering", "Electrical Engineering", "ELECTRICAL ENGINEERING", 
    "mechanical engineering", "Mechanical Engineering", "MECHANICAL ENGINEERING", 
    "civil engineering", "Civil Engineering", "CIVIL ENGINEERING", 
    "chemical engineering", "Chemical Engineering", "CHEMICAL ENGINEERING", 
    "high school", "High School", "HIGH SCHOOL", 
    "college", "College", "COLLEGE", 
    "university", "University", "UNIVERSITY", 
    "academy", "Academy", "ACADEMY", 
    "institute", "Institute", "INSTITUTE", 
    "majored in", "Majored in", "MAJORED IN", 
    "degree in", "Degree in", "DEGREE IN", 
    "field of study", "Field of Study", "FIELD OF STUDY", 
    "course", "Course", "COURSE", 
    "specialization", "Specialization", "SPECIALIZATION"
]


In [9]:
LANGUAGE_TERMS = [
    "english", "English", "ENGLISH", 
    "spanish", "Spanish", "SPANISH", "español", "Español", "ESPAÑOL",
    "french", "French", "FRENCH", "français", "Français", "FRANÇAIS",
    "german", "German", "GERMAN", "deutsch", "Deutsch", "DEUTSCH",
    "chinese", "Chinese", "CHINESE", 
    "mandarin", "Mandarin", "MANDARIN", 
    "cantonese", "Cantonese", "CANTONESE", 
    "simplified chinese", "Simplified Chinese", "SIMPLIFIED CHINESE", 
    "traditional chinese", "Traditional Chinese", "TRADITIONAL CHINESE", 
    "japanese", "Japanese", "JAPANESE", "日本語", "Nihongo", "NIHONGO", 
    "korean", "Korean", "KOREAN", "한국어", "Hangul", "HANGUL", "KOREAN", 
    "italian", "Italian", "ITALIAN", "italiano", "Italiano", "ITALIANO", 
    "portuguese", "Portuguese", "PORTUGUESE", "português", "Português", "PORTUGUÊS",
    "russian", "Russian", "RUSSIAN", "русский", "Russkiy", "RUSSKIY",
    "arabic", "Arabic", "ARABIC", "العربية", "Arabiya", "ARABIYA", 
    "hindi", "Hindi", "HINDI", "हिन्दी", "Hindī", "HINDĪ", 
    "bengali", "Bengali", "BENGALI", "বাংলা", "Bangla", "BANGLA", 
    "urdu", "Urdu", "URDU", "اردو", "Urdu", "URDU", 
    "punjabi", "Punjabi", "PUNJABI", "ਪੰਜਾਬੀ", "Punjabi", "PUNJABI", 
    "gujarati", "Gujarati", "GUJARATI", "ગુજરાતી", "Gujarati", "GUJARATI", 
    "polish", "Polish", "POLISH", "polski", "Polski", "POLSKI", 
    "dutch", "Dutch", "DUTCH", "nederlands", "Nederlands", "NEDERLANDS", 
    "greek", "Greek", "GREEK", "ελληνικά", "Elliniká", "ELLINIKÁ", 
    "swedish", "Swedish", "SWEDISH", "svenska", "Svenska", "SVENSKA", 
    "turkish", "Turkish", "TURKISH", "türkçe", "Türkçe", "TÜRKÇE", 
    "vietnamese", "Vietnamese", "VIETNAMESE", "Tiếng Việt", "TIẾNG VIỆT", "viet", "Viet", "VIET",
    "thai", "Thai", "THAI", "ไทย", "Thai", "THAI", 
    "hebrew", "Hebrew", "HEBREW", "עִבְרִית", "Ivrit", "IVRIT", 
    "swahili", "Swahili", "SWAHILI", "Kiswahili", "KISWAHILI", 
    "tamil", "Tamil", "TAMIL", "தமிழ்", "Tamil", "TAMIL", 
    "telugu", "Telugu", "TELUGU", "తెలుగు", "Telugu", "TELUGU", 
    "malayalam", "Malayalam", "MALAYALAM", "മലയാളം", "Malayāḷaṁ", "MALAYĀḶAṀ", 
    "marathi", "Marathi", "MARATHI", "मराठी", "Marāṭhī", "MARĀṬHĪ", 
    "persian", "Persian", "PERSIAN", "farsi", "Farsi", "FARSI", "فارسی", "Persian (Farsi)", "PERSIAN (FARSI)", 
    "tagalog", "Tagalog", "TAGALOG", 
    "indonesian", "Indonesian", "INDONESIAN", "bahasa indonesia", "Bahasa Indonesia", "BAHASA INDONESIA", 
    "malay", "Malay", "MALAY", "bahasa melayu", "Bahasa Melayu", "BAHASA MELAYU", 
    "romanian", "Romanian", "ROMANIAN", "Română", "ROMÂNĂ", 
    "czech", "Czech", "CZECH", "čeština", "Čeština", "ČEŠTINA", 
    "hungarian", "Hungarian", "HUNGARIAN", "magyar", "Magyar", "MAGYAR", 
    "slovak", "Slovak", "SLOVAK", "slovenčina", "Slovenčina", "SLOVENČINA", 
    "finnish", "Finnish", "FINNISH", "suomi", "Suomi", "SUOMI", 
    "norwegian", "Norwegian", "NORWEGIAN", "norsk", "Norsk", "NORSK", 
    "danish", "Danish", "DANISH", "dansk", "Dansk", "DANSK"
]


In [10]:
PROFICIENCY_LEVELS = [
    "a1", "A1","a 1", "A 1", "a-1", "A-1",
    "a2", "A2", "a 2", "A 2", "a-2", "A-2",
    "b1", "B1", "b 2", "B 2", "b-1", "B-1",
    "b2", "B2", "b 2", "B 2", "b-2", "B-2",
    "c1", "C1", "c 1", "C 1", "c-1", "C-1",
    "c2", "C2", "c 2", "C 2", "c-2", "C-2",
    "basic", "Basic", "BASIC", 
    "intermediate", "Intermediate", "INTERMEDIATE",
    "upper intermediate", "Upper intermediate", "UPPER INTERMEDIATE", 
    "upper-intermediate", "Upper-Intermediate", "UPPER-INTERMEDIATE", 
    "upperintermediate", "UpperIntermediate", "UPPERINTERMEDIATE",
    "upper-inetermediate", "Upper-Inetermediate", "UPPER-INETERMEDIATE", 
    "upper-internate", "Upper-Internate", "UPPER-INTERNATE",  
    "advanced", "Advanced", "ADVANCED",
    "native", "Native", "NATIVE",
    "professional", "Professional", "PROFESSIONAL",
    "ielts", "IELTS", "Ielts", 
    "toefl", "TOEFL", "Toefl"
]

In [11]:
def extract_skills(document):
    found_skills = set()
    for line in document:
        words = line.split()
        for word in words:
            if word.lower() in SKILL_TERMS:
                found_skills.add(word.lower())
        for i in range(len(words) - 1):
            two_word_skill = f"{words[i].lower()} {words[i+1].lower()}"
            if two_word_skill in SKILL_TERMS:
                found_skills.add(two_word_skill)
    return list(found_skills)

In [12]:
def extract_education(document):
    education = set()  
    for line in document:
        for term in EDUCATION_TERMS:
            if term.lower() in line:
                education.add(line.strip())
    return list(education)

In [13]:
def extract_languages(document):
    languages = {}
    language_pattern = re.compile(r'\b(' + '|'.join(LANGUAGE_TERMS) + r')\b', re.IGNORECASE)
    level_pattern = re.compile(r'\b(' + '|'.join(PROFICIENCY_LEVELS) + r')\b', re.IGNORECASE)
    for line in document:
        normalized_line = re.sub(r'\s+', ' ', line).lower().strip()
        for language_match in language_pattern.finditer(normalized_line):
            language = language_match.group(0).capitalize()
            level_match = level_pattern.search(normalized_line)
            if level_match:
                level = level_match.group(0).capitalize()
                languages[language] = level
            else:
                languages[language] = "Not specified"
    return languages

In [14]:
def extract_phone_numbers(document):
    phone_regex = re.compile(r'\b\d{10}\b|\(\d{3}\)\s*\d{3}[-\s]?\d{4}|\d{3}[-\s]?\d{3}[-\s]?\d{4}')
    phone_numbers = set()
    for line in document:
        found_numbers = phone_regex.findall(line)
        phone_numbers.update(found_numbers)
    return list(phone_numbers)

In [15]:
def extract_experience(document):
    experience_patterns = [
        re.compile(r'((jan(uary)?|feb(ruary)?|mar(ch)?|apr(il)?|may|jun(e)?|jul(y)?|aug(ust)?|sep(tember)?|oct(ober)?|nov(ember)?|dec(ember)?)\s+\d{4}\s*[-–]\s*(present|now|\d{4}))', re.IGNORECASE),
        re.compile(r'(\b\d{4}\b\s*[-–]\s*\b\d{4}\b|\b\d{4}\b\s*[-–]\s*(present|now))', re.IGNORECASE)
    ]
    experience = []
    current_experience = []
    for line in document:
        line_cleaned = line.strip()
        for pattern in experience_patterns:
            if pattern.search(line_cleaned):
                if current_experience:
                    experience.append(' '.join(current_experience))
                    current_experience = []
                current_experience.append(line_cleaned)
                break
        else:
            if current_experience:
                current_experience.append(line_cleaned)
    if current_experience:
        experience.append(' '.join(current_experience))
    return experience

In [16]:
def clean_text(text):
    ILLEGAL_CHARACTERS_RE = re.compile(r'[\x00-\x1F\x7F-\x9F]')
    return ILLEGAL_CHARACTERS_RE.sub('', text)

In [17]:
def main():
    phone_numbers_list = []
    skills_list = []
    experience_list = []
    education_list = []
    languages_list = []
    for file_name in os.listdir(RESUME_DIR):
        file_path = os.path.join(RESUME_DIR, file_name)
        if file_name.endswith('.pdf'):
            document = load_pdf_text(file_path)
        elif file_name.endswith('.docx'):
            document = load_docx_text(file_path)
        document = preprocess_text(document)
        phone_numbers = extract_phone_numbers(document)
        skills = extract_skills(document)
        experience = extract_experience(document)
        education = extract_education(document)
        languages = extract_languages(document)
        phone_numbers_list.append(clean_text(phone_numbers[0]) if phone_numbers else '')
        skills_list.append(clean_text(', '.join(skills)) if skills else '')
        experience_list.append(clean_text(', '.join(experience)) if experience else '')
        education_list.append(clean_text(', '.join(education)) if education else '')
        languages_list.append(clean_text(', '.join([f"{lang} - {level}" for lang, level in languages.items()])) if languages else '')
    df = pd.DataFrame({
        'Phone Numbers': phone_numbers_list,
        'Skills': skills_list,
        'Experience': experience_list,
        'Education': education_list,
        'Languages': languages_list
    })    
    try:
        df.to_csv('/kaggle/working/extracted_data.csv', index=False)
        print("Extraction complete. Data saved to extracted_data.csv.")
    except Exception as e:
        print(f"Failed to save the CSV file: {e}")

if __name__ == "__main__":
    main()

Extraction complete. Data saved to extracted_data.csv.


In [18]:
def display_resume_details(file_name):
    file_path = os.path.join(RESUME_DIR, file_name)
    if file_name.endswith('.pdf'):
        document = load_pdf_text(file_path)
    elif file_name.endswith('.docx'):
        document = load_docx_text(file_path)
    document = preprocess_text(document)
    phone_numbers = extract_phone_numbers(document)
    skills = extract_skills(document)
    experience = extract_experience(document)
    education = extract_education(document)
    languages = extract_languages(document)
    print("\n" + "="*80)
    print(f"\033[1mResume: {file_name}\033[0m".center(80))
    print("="*80)
    print(f"\n\033[1mPhone Numbers:\033[0m")
    if phone_numbers:
        for number in phone_numbers:
            print(f"  - {number}")
    else:
        print("  - No phone numbers found.")
    print(f"\n\033[1mSkills:\033[0m")
    if skills:
        for skill in skills:
            print(f"  - {skill.title()}")
    else:
        print("  - No skills found.")
    print(f"\n\033[1mExperience:\033[0m")
    if experience:
        for exp in experience:
            print(f"  - {exp}")
    else:
        print("  - No experience details found.")
    print(f"\n\033[1mEducation:\033[0m")
    if education:
        for edu in education:
            print(f"  - {edu.title()}")
    else:
        print("  - No education details found.")
    print(f"\n\033[1mLanguages:\033[0m")
    if languages:
        for lang, level in languages.items():
            print(f"  - {lang.title()} - {level.capitalize()}")
    else:
        print("  - No languages found.")    
    print("="*80 + "\n")

In [19]:
# Example Usage for Displaying Details
for resume in os.listdir(RESUME_DIR):
    display_resume_details(resume)


                         Resume: resume_16.pdf                          

Phone Numbers:
  - 9008036407

Skills:
  - Unit Testing
  - Html
  - Problem-Solving
  - Css
  - Js
  - Oracle
  - Integration Testing
  - Javascript
  - Java
  - Sql
  - Sql Server
  - Communication
  - Server Management
  - Angular
  - Mysql
  - Jquery
  - Agile

Experience:
  - No experience details found.

Education:
  - Databases (Less Than 1 Year) Eclipse (Less Than 1 Year)
  - Amrita School Of Engineering - Bengaluru Karnataka
  - Additional Information
  - • Description: Full Stack Web Development With Heavy Emphasis On Front End Responsibilities Include
  - Closely With Project Managers And Leads In Agile Environment
  - Echnologies: Jdbc
  - Aishveryaagarwal7_Krk@Indeedemailcom
  - Ools: Eclipse Ms Sql Server Management
  - ➢ Project: Hobs Product
  - For The Owned Use Cases Creating Database And Updating As Per The Requirement Unit Testing And
  - Modifying Existing Programs With Enhancements Developi

In [20]:
def display_resume_details_from_path(file_path):
    file_name = os.path.basename(file_path)
    if file_name.endswith('.pdf'):
        document = load_pdf_text(file_path)
    document = preprocess_text(document)
    phone_numbers = extract_phone_numbers(document)
    skills = extract_skills(document)
    experience = extract_experience(document)
    education = extract_education(document)
    languages = extract_languages(document)
    print(f"\n\033[1mResume: {file_name}\033[0m")
    print("="*60)    
    print(f"\033[1mPhone Numbers:\033[0m")
    if phone_numbers:
        for number in phone_numbers:
            print(f"  - {number}")
    else:
        print("  - No phone numbers found.")    
    print(f"\n\033[1mSkills:\033[0m")
    if skills:
        for skill in skills:
            print(f"  - {skill.title()}")
    else:
        print("  - No skills found.")    
    print(f"\n\033[1mExperience:\033[0m")
    if experience:
        for exp in experience:
            print(f"  - {exp}")
    else:
        print("  - No experience details found.")    
    print(f"\n\033[1mEducation:\033[0m")
    if education:
        for edu in education:
            print(f"  - {edu.title()}")
    else:
        print("  - No education details found.")  
    print(f"\n\033[1mLanguages:\033[0m")
    if languages:
        for lang, level in languages.items():
            print(f"  - {lang} - {level}")
    else:
        print("  - No languages found.")
    
    print("="*60)

In [21]:
resume_path = '/kaggle/input/resume-samples/Pejman-Rafati-Moghadam-V8.pdf' 
display_resume_details_from_path(resume_path)


Resume: Pejman-Rafati-Moghadam-V8.pdf
Phone Numbers:
  - 911 989 8709

Skills:
  - Configuration
  - Voip
  - Ticket System
  - Network Monitoring
  - Windows
  - Fluke Tests
  - Linux
  - Laptops
  - Kaspersky
  - Firewalls
  - Network Infrastructure
  - Virtualization
  - Surveillance Systems
  - Servers
  - Macos
  - Software Troubleshooting
  - Switches
  - Winbox
  - Printers

Experience:
  - oct 2021 - present (3y) installation configuration and maintenance of +3 servers +500 laptops +50 printers troubleshooting and network monitoring ensuring a timely resolution and minimizing downtime resulting in 45% reduction in overall downtime for +500 users implemented and configured various +500 software applications and +100 operating systems provided remote technical support via remote desktop tools manager: amid kalmarzi moghaddam – amidkalmarzi@gmailcom winbox kaspersky ticket system (familiarity) it support specialist oct 2020 - oct 2 021 (1y) troubleshoot hardware and software issu

In [22]:
resume_path = '/kaggle/input/resume-samples/Mansour-Mayahi-V7.pdf'  
display_resume_details_from_path(resume_path)


Resume: Mansour-Mayahi-V7.pdf
Phone Numbers:
  - 935 6566800

Skills:
  - Configuration
  - Network Monitoring
  - Vmware
  - Esxi
  - Printers
  - Windows
  - Server Virtualization
  - Server Management
  - Repairing Issues
  - Laptops
  - Firewalls
  - Administration
  - Network Security
  - Virtualization
  - Performance Optimization
  - Servers
  - Switches
  - Security Hardening
  - Diagnosing
  - Desktops

Experience:
  - sep 2016 - present  • managed the deployment upgrades and lifecycle of microsoft windows servers and physical servers • installed and configured routers switches and cabling including server room setup • deployed and managed virtual machines including integrated virtual network configurations • installed configured and managed esxi environments for server virtualization • implemented data backup and recovery strategies ensuring minimal data loss and quick recovery • reduced pc setup time by over 50% using os image creation and restoration • managed a network of

In [23]:
resume_path = '/kaggle/input/resume-samples/Shima-Atefi-Tali-V5.pdf'  
display_resume_details_from_path(resume_path)


Resume: Shima-Atefi-Tali-V5.pdf
Phone Numbers:
  - 9891517313

Skills:
  - Configuration
  - Vmware
  - Esxi
  - Windows
  - Server Virtualization
  - Server Management
  - Mysql
  - Laptops
  - Firewalls
  - Network Security
  - Vcp-Dcv
  - Virtualization
  - Aws
  - Backup Tools
  - Python
  - Servers
  - Switches
  - Vmware Vcenter
  - Desktops

Experience:
  - 2006 - 2010 it engineering (bsc) isfahan university of technology mtcna certificate no 1611na924 2016 vcp-dcv certificate kian rayaneh institute iran 2018 “introduction to programming using python” arjang institute iran 2021 “advanced python programming” maktabkhoone institute iran 2022 “aws cloud practitioner essentials” coursera 2024 “aws cloud practitioner essentials” coursera 2024 english (ielts ac 70) german (a1) persian (native)

Education:
  - Creating Computer Network Courses Teaching The Employees Of Big Organizations And Holding Theorical And Technical Workshops
  - Telecommunication Engineering-Network (Msc)
  - “